In [ ]:
## analyze same analytics with other cascades(as with juul)..
## look at all other cascades...

In [346]:
import pandas as pd
import numpy as np
import os
import git
import ast
import matplotlib.pyplot as plt
import networkx as nx
from tqdm import tqdm_notebook as tqdm
import tweepy
from cascade import Cascade

In [347]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [348]:
%reload_ext autoreload

In [349]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)

In [350]:
model_path = os.path.join(get_git_root(os.getcwd()),"models")

In [351]:
def get_git_root(path):
    git_repo = git.Repo(path, search_parent_directories=True)
    git_root = git_repo.git.rev_parse("--show-toplevel")
    return git_root

In [352]:
def get_shells(G):
    nodes_depth = nx.get_node_attributes(G,'depth')
    if nodes_depth:
        counter = 0
        shells = list()
        while True:
            if ([k for k,v in nodes_depth.items() if v == counter]):
                shells.append([k for k,v in nodes_depth.items() if v == counter])
                counter += 1 
            else:
                break
        return(shells)
    else:
        print("depth not encoded in graph")
    

In [353]:
def get_color_map(pos,shells):
    depth_nodes = dict()
    color_map = list()
    for idx,nodes in enumerate(shells):
        depth_nodes[idx] = nodes
    color_list = ["yellow","green","blue","pink","violet"]
    for node in pos.keys():
        depth = [depth for depth,nodes in depth_nodes.items() if node in nodes][0]
        color_map.append(color_list[depth])  
    return(color_map)

In [354]:
def plot(G):
    %matplotlib inline
    plt.figure(figsize=(15,15))
    shells = get_shells(G)
    pos = nx.shell_layout(G,shells)
    color_map = get_color_map(pos,shells)
    nx.draw(G,with_labels=False,nodelist=pos.keys(),pos=pos,node_color=color_map)
    ax = plt.gca()
    ax.set_aspect('equal')
    ax.set_axis_off()
    plt.title("2 level cascade")
    plt.show()

In [357]:
hexagon_path = os.path.join(get_git_root(os.getcwd()),"input","hexagonData.csv")
model_path = os.path.join(get_git_root(os.getcwd()),"models")
graph_path = os.path.join(get_git_root(os.getcwd()),"models","graphs")

In [356]:
hexagon_data = pd.read_csv(hexagon_path,lineterminator="\n")

In [ ]:
G_tattoo = nx.read_gpickle(os.path.join(model_path,"tattoo.gpickle"))
G_juul = nx.read_gpickle(os.path.join(model_path,"juul_cascade.gpickle"))
G_cascade3 = nx.read_gpickle(os.path.join(model_path,"cascade3.gpickle"))

In [ ]:
plot(G_tattoo)

In [ ]:
## get all of the cascades..
hexagon_data.loc[hexagon_data['retweetCount'] >= 1]

In [365]:
## getting the tweet text and their occurances
tweet_text_list = list()
df_tweets = pd.DataFrame([])
for index,tweet in hexagon_data.iterrows():
    user = tweet['userID']
    text = tweet['tweetText']
    retweet_count = tweet['retweetCount']
    if retweet_count > 0:
        if text not in tweet_text_list:
            tweet_text_list.append(text)
            df_tweets = df_tweets.append(pd.DataFrame({'tweet_text':text, 
                                           'retweet_count':retweet_count},index=[0]),
                             ignore_index=True)

In [ ]:
## there are 1771 cascades = 0
len(df_tweets.loc[df_tweets.retweet_count ==0])

In [345]:
## there are 1771 cascades > 0
len(df_tweets.loc[df_tweets.retweet_count >0])

1771

In [372]:
len(df_tweets.loc[df_tweets.retweet_count >=2])

751

In [ ]:
(df_tweets.loc[df_tweets.retweet_count >10])

In [269]:
hexagon_data

,favourites_count,friendList,friendsCount,hashtags,imageurl,lang,retweetCount,retweeted,tweetCreatedAt,tweetId,tweetText,userCreatedAt,userDescription,userFollowersCount,userID,userLocation,userName
0,668,None,371,['JUUL'],http://pbs.twimg.com/profile_images/590395947990929408/UwjBNF2R_normal.jpg,en,0,False,2015-06-10 00:25:21,608429710989783041,@KenHeindel Sorry to hear that Ken. Please DM us your email and the #JUUL care team will assist however they can.,2014-12-03 23:28:22,"At JUUL, our mission is to improve the lives of the world’s one billion adult smokers. +21+ DM for Twitter Customer Service 8am-5:30pm PT M-F. #SwitchToJUUL",19707,2918018660,San Francisco,JUUL\r
1,4668,None,881,['JUUL'],http://pbs.twimg.com/profile_images/779839998480658432/a0lADx7M_normal.jpg,en,0,False,2015-06-04 21:57:23,606580531233316864,#JUUL - Smoking Evolved. Check it out: https://t.co/0e6eH5noYm,2012-02-06 23:31:52,"Desert Shield/Storm Veteran, Ex stockbroker, ex-smoker, wannabe Transhumanist",189,485212274,"North Carolina, USA",Thomas Toledo\r
2,668,None,371,['JUUL'],http://pbs.twimg.com/profile_images/590395947990929408/UwjBNF2R_normal.jpg,en,0,False,2015-06-08 21:33:09,608023985494720512,@katsushiro Nothing like that planned for the near future. Be sure you're signed up for all the latest from #JUUL: https://t.co/CKtxfeZmJg,2014-12-03 23:28:22,"At JUUL, our mission is to improve the lives of the world’s one billion adult smokers. +21+ DM for Twitter Customer Service 8am-5:30pm PT M-F. #SwitchToJUUL",19707,2918018660,San Francisco,JUUL\r
3,904,None,483,"['pride', 'JUULvapor']",http://pbs.twimg.com/profile_images/989187403465285632/qHg7Zee7_normal.jpg,en,0,False,2015-06-29 15:58:19,615549868967337984,Last one I promise 😜🌈 #pride #JUULvapor samboudloche https://t.co/UYk3uN4CPV,2009-04-04 05:27:45,It's GUTCHES like DUCHESS not GOOCHES like HOOCHES...the more you know!,522,28747090,"New York, NY",Kevin Gutches\r
4,668,None,371,['JUUL'],http://pbs.twimg.com/profile_images/590395947990929408/UwjBNF2R_normal.jpg,en,0,False,2015-06-22 23:08:37,613121440116707329,@jacksonfall #JUUL is pretty great - you can pick one up on our website :-),2014-12-03 23:28:22,"At JUUL, our mission is to improve the lives of the world’s one billion adult smokers. +21+ DM for Twitter Customer Service 8am-5:30pm PT M-F. #SwitchToJUUL",19707,2918018660,San Francisco,JUUL\r
5,0,None,61,['juul'],http://pbs.twimg.com/profile_images/1595792899/IMG_1020_normal.jpg,en,0,False,2015-06-05 00:30:00,606618942061330432,"At Juul event with guillaumemalheiro #juul @ Chelsea, nyc https://t.co/7vWtDMFqCF",2009-07-07 15:51:27,Globe trotter,361,54585605,New York,PANNIER Damien\r
6,10,None,34,"['PAX', 'JUUL', 'vape']",http://pbs.twimg.com/profile_images/536708664380239872/TN0lNF8a_normal.png,et,0,False,2015-06-06 18:16:00,607249596478324736,RT xpressingit: Last one I promise juulvapor paxvapor #PAX #JUUL #vape sarahrinsf rpmumby sup___dude… http://t.co/eHQXE2TfGc;,2014-11-24 00:57:22,The Worlds Best #Vape Store:,3055,2908532184,NaN,Vape Merch\r
7,0,None,61,"['juul', 'jackstudio']",http://pbs.twimg.com/profile_images/1595792899/IMG_1020_normal.jpg,en,0,False,2015-06-05 01:45:26,606637925762277377,You rock @iamamayzing !! Thank you for invinting us 😊😊#juul #jackstudio @ Jack Studios https://t.co/eyTFZdpZvD,2009-07-07 15:51:27,Globe trotter,361,54585605,New York,PANNIER Damien\r
8,668,None,371,['JUUL'],http://pbs.twimg.com/profile_images/590395947990929408/UwjBNF2R_normal.jpg,en,0,False,2015-06-03 18:48:23,606170580732256256,@pkim24 Cheers! Be sure to sign up for 25% off your first #JUUL. Can't wait to hear what you think.,2014-12-03 23:28:22,"At JUUL, our mission is to improve the lives of the world’s one billion adult smokers. +21+ DM for Twitter Customer Service 8am-5:30pm PT M-F. #SwitchToJUUL",19707,2918018660,San Francisco,JUUL\r
9,3203,None,487,['JUUL'],http://pbs.twimg.com/profile_images/1044921882292756480/sZPO_PQJ_normal.jpg,en,0,False,2015-06-13 02:04:00,609541699442999296,Check me 

In [344]:
cas = Cascade()
for i in range(len(df_tweets[2:5])):
    cascade = hexagon_data.loc[hexagon_data.tweetText == df_tweets.tweet_text[i]]
    cascade['tweetCreatedAt'] = pd.to_datetime(cascade['tweetCreatedAt'])
    cascade.sort_values(by='tweetCreatedAt',ascending = True, inplace=True)
    source_node = cascade.head(1)['userID'].values[0]
    retweet_count = cascade.head(1)['retweetCount'].values[0]
    users = list(cascade['userID'])
    users.remove(source_node)
    G =  cas.get_cascade(cascade,source_node,users,level_termiante=None)
    filename = str('G_' + str(source_node) + '_'+ str(retweet_count)+'.gpickle')
    if G:
        nx.write_gpickle(G,os.path.join(model_path,'graphs',filename))
    else:
        print("blank_graph")

/Users/akshayuppal/anaconda3/envs/juul/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/akshayuppal/anaconda3/envs/juul/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
  0%|          | 0/7 [00:00<?, ?it/s]

blank_graph


  0%|          | 0/5 [00:00<?, ?it/s]

finding connection for followers network might take some time



100%|██████████| 5/5 [00:00<00:00, 528.93it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
  0%|          | 0/1 [00:00<?, ?it/s]

at level 1
finding connection for followers network might take some time

at level 2
breaking as no progression


100%|██████████| 1/1 [00:00<00:00,  3.67it/s]

blank_graph


In [392]:
# lets read all the cascades and get some metrics
import os
filenames = [file for file in os.listdir(os.path.join(os.getcwd(),graph_path)) if file.endswith(".gpickle")]

In [397]:
counter = 0
for idx,file in enumerate(filenames):
    G = nx.read_gpickle(os.path.join(os.getcwd(),graph_path,file))
    if (G):
        print(idx,len(G.nodes()),file)

5 8 G_juul.gpickle
23 2 G_2762479215_2.gpickle
25 5 G_4855026431_5.gpickle
26 6 G_400465712_8.gpickle
41 2 G_364963104_11.gpickle
47 3 G_2266199337_6.gpickle
59 3 G_1093595864_26.gpickle
60 3 G_846461177500618752_6.gpickle
63 2 G_2764364902_15.gpickle
66 3 G_814313726513938432_4.gpickle
88 4 G_3434304875_3.gpickle
96 2 G_374100307_5.gpickle
108 4 G_2918018660_9.gpickle
133 2 G_854000327200309252_4.gpickle
136 2 G_151973643_4.gpickle
146 387 tattoo.gpickle
156 3 G_3.gpickle
157 2 G_1325161927_2.gpickle
162 2 G_22873437_25.gpickle
175 8 G_2937602802_8.gpickle
176 2 G_1950384332_3.gpickle
188 6 G_32125966_9.gpickle
189 2 G_2343297244_5.gpickle
195 6 G_16870853_7.gpickle
198 4 G_735989441009995776_5.gpickle
205 2 G_310016426_2.gpickle
225 4 G_1663551918_3.gpickle
240 4 G_2918018660_7.gpickle
244 5 G_3922284153_5.gpickle


In [385]:
G = nx.read_gpickle("/Users/akshayuppal/Desktop/thesis/twitter_juul/models/graphs/G_3.gpickle")

In [387]:
filenames

['G_837672001_1.gpickle',
 'G_842597397972770816_2.gpickle',
 'G_217598942_1.gpickle',
 'G_1613929088_1.gpickle',
 'G_2287391647_1.gpickle',
 'G_juul.gpickle',
 'G_271551073_2.gpickle',
 'G_2806223566_1.gpickle',
 'G_3973438023_1.gpickle',
 'G_400465712_1.gpickle',
 'G_1724518488_1.gpickle',
 'G_2918018660_2.gpickle',
 'G_348880104_2.gpickle',
 'G_21834914_1.gpickle',
 'G_3008733475_1.gpickle',
 'G_916714728_1.gpickle',
 'G_555997392_1.gpickle',
 'G_75653018_2.gpickle',
 'G_2865136112_1.gpickle',
 'G_1330608092_1.gpickle',
 'G_826269571497807872_1.gpickle',
 'G_384621252_1.gpickle',
 'G_38374327_1.gpickle',
 'G_2762479215_2.gpickle',
 'G_1881605982_1.gpickle',
 'G_4855026431_5.gpickle',
 'G_400465712_8.gpickle',
 'G_843524406882263040_1.gpickle',
 'G_2289012848_1.gpickle',
 'G_853411141_2.gpickle',
 '.DS_Store',
 'G_4131317897_1.gpickle',
 'G_636767319_4.gpickle',
 'G_2171281398_2.gpickle',
 'G_3125184862_1.gpickle',
 'G_1285016509_1.gpickle',
 'G_1102665744_1.gpickle',
 'G_1420901798_

In [382]:
os.path.join(os.getcwd(),graph_path,filenames[55])

'/Users/akshayuppal/Desktop/thesis/twitter_juul/models/graphs/G_2341669241_1.gpickle'

## testing

In [339]:
G= nx.read_gpickle("/Users/akshayuppal/Desktop/thesis/twitter_juul/models/graphs/G_16870853_7.gpickle")

In [340]:
G.nodes()

NodeView((15247690, 16870853, 34342490, 22212944, 423906777, 207413706))

In [343]:
G.nodes[22212944]

{'friends': 1396, 'followers': 313, 'level': 1}

In [330]:
cas = Cascade()
df_tweets = cas.get_unique_tweets(hexagon_data)